In [1]:
pwd

'C:\\Users\\RADHA\\Documents\\gentask'

In [2]:
!pip install glob2

In [3]:
!pip install pypdf

In [4]:
!pip install python-docx
!pip install PyPDF2
!pip install --upgrade PyPDF2


In [5]:
import os
import glob
from PyPDF2 import PdfReader
from docx import Document

def pdf_to_docx(pdf_path, docx_path):
    pdf_reader = PdfReader(pdf_path)
    document = Document()

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        document.add_paragraph(text)

    document.save(docx_path)

def convert_directory_pdf_to_docx(pdf_directory, docx_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(docx_directory):
        os.makedirs(docx_directory)

    # Get a list of PDF files in the input directory
    pdf_files = glob.glob(os.path.join(pdf_directory, '*.pdf'))

    for pdf_file in pdf_files:
        # Construct the output file path
        base_name = os.path.basename(pdf_file)
        file_name = os.path.splitext(base_name)[0]
        docx_file = os.path.join(docx_directory, f'{file_name}.docx')

        # Convert the PDF file to DOCX
        pdf_to_docx(pdf_file, docx_file)

# Example usage
pdf_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf'
docx_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf2docx'

convert_directory_pdf_to_docx(pdf_directory, docx_directory)


In [6]:
import os
import glob
from PyPDF2 import PdfReader
from docx import Document

def pdf_to_docx(pdf_path, docx_path):
    pdf_reader = PdfReader(pdf_path)
    document = Document()

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        document.add_paragraph(text)

    document.save(docx_path)

def convert_directory_pdf_to_docx(pdf_directory, docx_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(docx_directory):
        os.makedirs(docx_directory)

    # Get a list of PDF files in the input directory
    pdf_files = glob.glob(os.path.join(pdf_directory, '*.pdf'))

    total_files = len(pdf_files)
    print(f"Converting {total_files} PDF files to DOCX...")

    for idx, pdf_file in enumerate(pdf_files, 1):
        # Construct the output file path
        base_name = os.path.basename(pdf_file)
        file_name = os.path.splitext(base_name)[0]
        docx_file = os.path.join(docx_directory, f'{file_name}.docx')

        # Convert the PDF file to DOCX
        pdf_to_docx(pdf_file, docx_file)

        # Print the progress
        print(f"Processed file {idx}/{total_files}")

    print("Conversion completed successfully!")

# Example usage
pdf_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf'
docx_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf2docx'

convert_directory_pdf_to_docx(pdf_directory, docx_directory)


Converting 1 PDF files to DOCX...
Processed file 1/1
Conversion completed successfully!


In [7]:
import os
import glob
from PyPDF2 import PdfReader
from docx import Document
import re

def sanitize_text(text):
    # Remove unsupported characters
    sanitized_text = re.sub(r'[^\x00-\x7F]', '', text)
    return sanitized_text

def pdf_to_docx(pdf_path, docx_path):
    pdf_reader = PdfReader(pdf_path)
    document = Document()

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        text = page.extract_text()
        sanitized_text = sanitize_text(text)
        document.add_paragraph(sanitized_text)

    document.save(docx_path)

def convert_directory_pdf_to_docx(pdf_directory, docx_directory):
    # Create the output directory if it doesn't exist
    if not os.path.exists(docx_directory):
        os.makedirs(docx_directory)

    # Get a list of PDF files in the input directory
    pdf_files = glob.glob(os.path.join(pdf_directory, '*.pdf'))

    total_files = len(pdf_files)
    print(f"Converting {total_files} PDF files to DOCX...")

    for idx, pdf_file in enumerate(pdf_files, 1):
        # Construct the output file path
        base_name = os.path.basename(pdf_file)
        file_name = os.path.splitext(base_name)[0]
        docx_file = os.path.join(docx_directory, f'{file_name}.docx')

        # Convert the PDF file to DOCX
        pdf_to_docx(pdf_file, docx_file)

        # Print the progress
        print(f"Processed file {idx}/{total_files}")

    print("Conversion completed successfully!")

# Example usage
pdf_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf'
docx_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf2docx'

convert_directory_pdf_to_docx(pdf_directory, docx_directory)


Converting 1 PDF files to DOCX...
Processed file 1/1
Conversion completed successfully!


In [8]:
# import os
# import glob
# import docx
# import spacy
# from docx.oxml.ns import nsdecls
# from docx.oxml import parse_xml
# from PyPDF2 import PdfFileReader, PdfFileWriter

# # Load the custom NER model
# nlp = spacy.load("./model-last")

# def extract_custom_ner_entities(text):
#     """
#     Extract custom NER entities from text
#     """
#     doc = nlp(text)
#     custom_ner_entities = []
#     for ent in doc.ents:
#         custom_ner_entities.append((ent.label_, ent.text))
#     return custom_ner_entities

# def redact_matching_data(paragraph, matched_entities):
#     """
#     Redact matching entities in a paragraph
#     """
#     for entity_label, entity_text in matched_entities:
#         for run in paragraph.runs:
#             if entity_text in run.text:
#                 run_text = run.text.replace(entity_text, "[REDACTED]")
#                 run.clear()  # Remove the existing run
#                 new_run = paragraph.add_run(run_text)  # Create a new run with the redacted text
#                 font = new_run.font
#                 font.highlight_color = docx.enum.text.WD_COLOR_INDEX.BLACK  # Apply black highlight color for redacted text

# def redact_docx_file(input_file, output_file):
#     """
#     Redact custom NER entities in a DOCX file and save the redacted file
#     """
#     doc = docx.Document(input_file)

#     for paragraph in doc.paragraphs:
#         text = paragraph.text
#         matched_entities = extract_custom_ner_entities(text)
#         redact_matching_data(paragraph, matched_entities)

#     doc.save(output_file)

# def redact_pdf_file(input_file, output_file):
#     """
#     Redact custom NER entities in a PDF file and save the redacted file
#     """
#     pdf_reader = PdfFileReader(input_file)
#     pdf_writer = PdfFileWriter()

#     for page_num in range(pdf_reader.numPages):
#         page = pdf_reader.getPage(page_num)
#         text = page.extract_text()
#         matched_entities = extract_custom_ner_entities(text)
#         redacted_text = text

#         for entity_label, entity_text in matched_entities:
#             redacted_text = redacted_text.replace(entity_text, "[REDACTED]")

#         page = page.extract_text().replace(text, redacted_text)
#         pdf_writer.addPage(page)

#     with open(output_file, 'wb') as output_pdf:
#         pdf_writer.write(output_pdf)

# def redact_files_in_directory(input_directory, output_directory):
#     """
#     Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
#     """
#     os.makedirs(output_directory, exist_ok=True)
#     files = glob.glob(os.path.join(input_directory, "*.*"))

#     for file in files:
#         file_ext = os.path.splitext(file)[1].lower()
#         file_name = os.path.basename(file)
#         output_file = os.path.join(output_directory, file_name)

#         if file_ext == ".docx":
#             redact_docx_file(file, output_file)
#             print(f"Redacted DOCX file saved: {output_file}")
#         elif file_ext == ".pdf":
#             redact_pdf_file(file, output_file)
#             print(f"Redacted PDF file saved: {output_file}")

# # Specify the input and output directories
# input_directory = '/home/ec2-user/SageMaker/synthetic_resumes3/'
# output_directory = '/home/ec2-user/SageMaker/syn_redact/'

# # Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
# redact_files_in_directory(input_directory, output_directory)


In [9]:
# import os
# import glob
# import docx
# import spacy
# from docx.oxml.ns import nsdecls
# from docx.oxml import parse_xml
# from PyPDF4 import PdfFileReader, PdfFileWriter

# # Load the custom NER model
# nlp = spacy.load("./model-last")

# def extract_custom_ner_entities(text):
#     """
#     Extract custom NER entities from text
#     """
#     doc = nlp(text)
#     custom_ner_entities = []
#     for ent in doc.ents:
#         custom_ner_entities.append((ent.label_, ent.text))
#     return custom_ner_entities

# def redact_matching_data(paragraph, matched_entities):
#     """
#     Redact matching entities in a paragraph
#     """
#     for entity_label, entity_text in matched_entities:
#         for run in paragraph.runs:
#             if entity_text in run.text:
#                 run_text = run.text.replace(entity_text, "[REDACTED]")
#                 run.clear()  # Remove the existing run
#                 new_run = paragraph.add_run(run_text)  # Create a new run with the redacted text
#                 font = new_run.font
#                 font.highlight_color = docx.enum.text.WD_COLOR_INDEX.BLACK  # Apply black highlight color for redacted text

# def redact_docx_file(input_file, output_file):
#     """
#     Redact custom NER entities in a DOCX file and save the redacted file
#     """
#     doc = docx.Document(input_file)

#     for paragraph in doc.paragraphs:
#         text = paragraph.text
#         matched_entities = extract_custom_ner_entities(text)
#         redact_matching_data(paragraph, matched_entities)

#     doc.save(output_file)

# def redact_pdf_file(input_file, output_file):
#     """
#     Redact custom NER entities in a PDF file and save the redacted file
#     """
#     pdf_reader = PdfFileReader(input_file)
#     pdf_writer = PdfFileWriter()

#     for page_num in range(pdf_reader.numPages):
#         page = pdf_reader.getPage(page_num)
#         text = page.extractText()
#         matched_entities = extract_custom_ner_entities(text)
#         redacted_text = text

#         for entity_label, entity_text in matched_entities:
#             redacted_text = redacted_text.replace(entity_text, "[REDACTED]")

#         page = page.extractText().replace(text, redacted_text)
#         pdf_writer.addPage(page)

#     with open(output_file, 'wb') as output_pdf:
#         pdf_writer.write(output_pdf)

# def redact_files_in_directory(input_directory, output_directory):
#     """
#     Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
#     """
#     os.makedirs(output_directory, exist_ok=True)
#     files = glob.glob(os.path.join(input_directory, "*.*"))

#     for file in files:
#         file_ext = os.path.splitext(file)[1].lower()
#         file_name = os.path.basename(file)
#         output_file = os.path.join(output_directory, file_name)

#         if file_ext == ".docx":
#             redact_docx_file(file, output_file)
#             print(f"Redacted DOCX file saved: {output_file}")
#         elif file_ext == ".pdf":
#             redact_pdf_file(file, output_file)
#             print(f"Redacted PDF file saved: {output_file}")

# # Specify the input and output directories
# input_directory = '/home/ec2-user/SageMaker/synthetic_resumes3/'
# output_directory = '/home/ec2-user/SageMaker/syn_redact/'

# # Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
# redact_files_in_directory(input_directory, output_directory)


In [10]:
# import os
# import glob
# import docx
# import spacy
# from docx.oxml.ns import nsdecls
# from docx.oxml import parse_xml
# from PyPDF4 import PdfFileReader, PdfFileWriter

# # Load the custom NER model
# nlp = spacy.load("./model-last")

# def extract_custom_ner_entities(text):
#     """
#     Extract custom NER entities from text
#     """
#     doc = nlp(text)
#     custom_ner_entities = []
#     for ent in doc.ents:
#         custom_ner_entities.append((ent.label_, ent.text))
#     return custom_ner_entities

# def redact_matching_data(paragraph, matched_entities):
#     """
#     Redact matching entities in a paragraph
#     """
#     for entity_label, entity_text in matched_entities:
#         for run in paragraph.runs:
#             if entity_text in run.text:
#                 run_text = run.text.replace(entity_text, "[REDACTED]")
#                 run.clear()  # Remove the existing run
#                 new_run = paragraph.add_run(run_text)  # Create a new run with the redacted text
#                 font = new_run.font
#                 font.highlight_color = docx.enum.text.WD_COLOR_INDEX.BLACK  # Apply black highlight color for redacted text

# def redact_docx_file(input_file, output_file):
#     """
#     Redact custom NER entities in a DOCX file and save the redacted file
#     """
#     doc = docx.Document(input_file)

#     for paragraph in doc.paragraphs:
#         text = paragraph.text
#         matched_entities = extract_custom_ner_entities(text)
#         redact_matching_data(paragraph, matched_entities)

#     doc.save(output_file)

# def redact_pdf_file(input_file, output_file):
#     """
#     Redact custom NER entities in a PDF file and save the redacted file
#     """
#     pdf_reader = PdfFileReader(input_file)
#     pdf_writer = PdfFileWriter()

#     for page_num in range(pdf_reader.getNumPages()):
#         page = pdf_reader.getPage(page_num)
#         text = page.extractText()
#         matched_entities = extract_custom_ner_entities(text)
#         redacted_text = text

#         for entity_label, entity_text in matched_entities:
#             redacted_text = redacted_text.replace(entity_text, "[REDACTED]")

#         page.mergePage(redacted_text)
#         pdf_writer.addPage(page)

#     with open(output_file, 'wb') as output_pdf:
#         pdf_writer.write(output_pdf)

# def redact_files_in_directory(input_directory, output_directory):
#     """
#     Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
#     """
#     os.makedirs(output_directory, exist_ok=True)
#     files = glob.glob(os.path.join(input_directory, "*.*"))

#     for file in files:
#         file_ext = os.path.splitext(file)[1].lower()
#         file_name = os.path.basename(file)
#         output_file = os.path.join(output_directory, file_name)

#         if file_ext == ".docx":
#             redact_docx_file(file, output_file)
#             print(f"Redacted DOCX file saved: {output_file}")
#         elif file_ext == ".pdf":
#             redact_pdf_file(file, output_file)
#             print(f"Redacted PDF file saved: {output_file}")

# # Specify the input and output directories
# input_directory = '/home/ec2-user/SageMaker/synthetic_resumes3/'
# output_directory = '/home/ec2-user/SageMaker/syn_redact/'

# # Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
# redact_files_in_directory(input_directory, output_directory)


In [11]:
# import os
# import glob
# import docx
# import spacy
# from PyPDF4 import PdfFileReader, PdfFileWriter

# # Load the custom NER model
# nlp = spacy.load("./model-last")

# def extract_custom_ner_entities(text):
#     """
#     Extract custom NER entities from text
#     """
#     doc = nlp(text)
#     custom_ner_entities = []
#     for ent in doc.ents:
#         custom_ner_entities.append((ent.label_, ent.text))
#     return custom_ner_entities

# def redact_matching_data(paragraph, matched_entities):
#     """
#     Redact matching entities in a paragraph
#     """
#     for entity_label, entity_text in matched_entities:
#         for run in paragraph.runs:
#             if entity_text in run.text:
#                 run_text = run.text.replace(entity_text, "[REDACTED]")
#                 run.clear()  # Remove the existing run
#                 new_run = paragraph.add_run(run_text)  # Create a new run with the redacted text
#                 font = new_run.font
#                 font.highlight_color = docx.enum.text.WD_COLOR_INDEX.BLACK  # Apply black highlight color for redacted text

# def redact_docx_file(input_file, output_file):
#     """
#     Redact custom NER entities in a DOCX file and save the redacted file
#     """
#     doc = docx.Document(input_file)

#     for paragraph in doc.paragraphs:
#         text = paragraph.text
#         matched_entities = extract_custom_ner_entities(text)
#         redact_matching_data(paragraph, matched_entities)

#     doc.save(output_file)

# def redact_pdf_file(input_file, output_file):
#     """
#     Redact custom NER entities in a PDF file and save the redacted file
#     """
#     pdf_reader = PdfFileReader(input_file)
#     pdf_writer = PdfFileWriter()

#     for page_num in range(pdf_reader.getNumPages()):
#         page = pdf_reader.getPage(page_num)
#         text = page.extractText()
#         matched_entities = extract_custom_ner_entities(text)
#         redacted_text = text

#         for entity_label, entity_text in matched_entities:
#             redacted_text = redacted_text.replace(entity_text, "[REDACTED]")

#         page.mergePage(redacted_text)
#         pdf_writer.addPage(page)

#     with open(output_file, 'wb') as output_pdf:
#         pdf_writer.write(output_pdf)

# def redact_files_in_directory(input_directory, output_directory):
#     """
#     Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
#     """
#     os.makedirs(output_directory, exist_ok=True)
#     files = glob.glob(os.path.join(input_directory, "*.*"))

#     for file in files:
#         file_ext = os.path.splitext(file)[1].lower()
#         file_name = os.path.basename(file)
#         output_file = os.path.join(output_directory, file_name)

#         if file_ext == ".docx":
#             text = read_docx_file(file)
#             matched_entities = extract_custom_ner_entities(text)
#             redacted_text = redact_text(text, matched_entities)
#             save_redacted_docx(output_file, redacted_text)
#             print(f"Redacted DOCX file saved: {output_file}")
#         elif file_ext == ".pdf":
#             text = read_pdf_file(file)
#             matched_entities = extract_custom_ner_entities(text)
#             redacted_text = redact_text(text, matched_entities)
#             save_redacted_pdf(output_file, redacted_text)
#             print(f"Redacted PDF file saved: {output_file}")

# def read_docx_file(file_path):
#     """
#     Read the content of a DOCX file
#     """
#     doc = docx.Document(file_path)
#     content = "\n".join([paragraph.text for paragraph in doc.paragraphs])
#     return content

# def read_pdf_file(file_path):
#     """
#     Read the content of a PDF file
#     """
#     with open(file_path, 'rb') as file:
#         pdf_reader = PdfFileReader(file)
#         content = ""
#         for page_num in range(pdf_reader.getNumPages()):
#             page = pdf_reader.getPage(page_num)
#             content += page.extract_text()
#     return content

# def redact_text(text, matched_entities):
#     """
#     Redact matching entities in the text
#     """
#     redacted_text = text
#     for entity_label, entity_text in matched_entities:
#         redacted_text = redacted_text.replace(entity_text, "[REDACTED]")
#     return redacted_text

# def save_redacted_docx(file_path, redacted_text):
#     """
#     Save the redacted text as a DOCX file
#     """
#     doc = docx.Document()
#     doc.add_paragraph(redacted_text)
#     doc.save(file_path)

# def save_redacted_pdf(file_path, redacted_text):
#     """
#     Save the redacted text as a PDF file
#     """
#     pdf_writer = PdfFileWriter()
#     pdf_writer.addPage(redacted_text)
#     with open(file_path, 'wb') as output_pdf:
#         pdf_writer.write(output_pdf)

# # Specify the input and output directories
# input_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf'
# output_directory = r'C:\Users\RADHA\Documents\gentask\samplepdf2docx'


# # Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
# redact_files_in_directory(input_directory, output_directory)


In [12]:
# import os
# import glob
# import docx
# import spacy
# import fitz

# # Load the custom NER model
# nlp = spacy.load("./model-last")

# def extract_custom_ner_entities(text):
#     """
#     Extract custom NER entities from text
#     """
#     doc = nlp(text)
#     custom_ner_entities = []
#     for ent in doc.ents:
#         custom_ner_entities.append((ent.label_, ent.text))
#     return custom_ner_entities

# def redact_matching_data(paragraph, matched_entities):
#     """
#     Redact matching entities in a paragraph
#     """
#     for entity_label, entity_text in matched_entities:
#         for run in paragraph.runs:
#             if entity_text in run.text:
#                 run_text = run.text.replace(entity_text, "[REDACTED]")
#                 run.clear()  # Remove the existing run
#                 new_run = paragraph.add_run(run_text)  # Create a new run with the redacted text
#                 font = new_run.font
#                 font.highlight_color = docx.enum.text.WD_COLOR_INDEX.BLACK  # Apply black highlight color for redacted text

# def redact_docx_file(input_file, output_file):
#     """
#     Redact custom NER entities in a DOCX file and save the redacted file
#     """
#     doc = docx.Document(input_file)

#     for paragraph in doc.paragraphs:
#         text = paragraph.text
#         matched_entities = extract_custom_ner_entities(text)
#         redact_matching_data(paragraph, matched_entities)

#     doc.save(output_file)

# def redact_pdf_file(input_file, output_file):
#     """
#     Redact custom NER entities in a PDF file and save the redacted file
#     """
#     doc = fitz.open(input_file)

#     for page_num in range(doc.page_count):
#         page = doc.load_page(page_num)
#         text = page.get_text()
#         matched_entities = extract_custom_ner_entities(text)
#         redacted_text = text

#         for entity_label, entity_text in matched_entities:
#             redacted_text = redacted_text.replace(entity_text, "[REDACTED]")

#         page.add_redact_annot(fitz.Rect(0, 0, page.rect.width, page.rect.height), text_color=(0, 0, 0))
#         page.apply_redactions()

#     doc.save(output_file)

# def redact_files_in_directory(input_directory, output_directory):
#     """
#     Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
#     """
#     os.makedirs(output_directory, exist_ok=True)
#     files = glob.glob(os.path.join(input_directory, "*.*"))

#     for file in files:
#         file_ext = os.path.splitext(file)[1].lower()
#         file_name = os.path.basename(file)
#         output_file = os.path.join(output_directory, file_name)

#         if file_ext == ".docx":
#             redact_docx_file(file, output_file)
#             print(f"Redacted DOCX file saved: {output_file}")
#         elif file_ext == ".pdf":
#             redact_pdf_file(file, output_file)
#             print(f"Redacted PDF file saved: {output_file}")

# # Specify the input and output directories
# input_directory = r'/content/drive/MyDrive/customPIIredaction-main/synthetic_resumes1'
# output_directory = r'/content/drive/MyDrive/customPIIredaction-main/synthetic_resumes1/samplepdf2docx'

# # Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
# redact_files_in_directory(input_directory, output_directory)

OSError: [E050] Can't find model './model-last'. It doesn't seem to be a Python package or a valid path to a data directory.

In [ ]:
#redact either pdf or docx

import os
import glob
import docx
import spacy
import fitz
from typing import Tuple
from io import BytesIO

# Specify the input and output directories
input_directory = r'/content/drive/MyDrive/customPIIredaction-main/synthetic_resumes1'
output_directory = r'/content/drive/MyDrive/customPIIredaction-main/synthetic_resumes1/samplepdf2docx'

# Load the custom NER model
nlp = spacy.load("./model-last")

def extract_custom_ner_entities(text):
    """
    Extract custom NER entities from text
    """
    doc = nlp(text)
    custom_ner_entities = []
    for ent in doc.ents:
        custom_ner_entities.append((ent.label_, ent.text))
    return custom_ner_entities

def redact_matching_data(page, matched_entities):
    """
    Redact matching entities in a page (PDF) or paragraph (DOCX)
    """
    matches_found = 0
    if isinstance(page, fitz.Page):  # For PDF files
        for entity_label, entity_text in matched_entities:
            matching_entity_area = page.search_for(entity_text)
            # Redact matching entities
            for area in matching_entity_area:
                page.add_redact_annot(area)
                matches_found += 1
        # Apply the redaction
        page.apply_redactions()
    elif isinstance(page, docx.text.paragraph.Paragraph):  # For DOCX files
        for entity_label, entity_text in matched_entities:
            if entity_text in page.text:
                page_text = page.text.replace(entity_text, "[REDACTED]")
                page.clear()  # Remove the existing text
                page.add_run(page_text)  # Add the redacted text
                matches_found += 1
    return matches_found

def process_data(input_file: str, output_file: str, pages: Tuple = None, action: str = 'Redact'):
    """
    Process the PDF or DOCX file and perform redaction
    """
    file_ext = os.path.splitext(input_file)[1].lower()

    if file_ext == ".pdf":
        # Open the PDF
        pdfDoc = fitz.open(input_file, filetype="pdf")
        # Save the generated PDF to memory buffer
        output_buffer = BytesIO()
        total_matches = 0
        # Iterate through pages
        for pg in range(pdfDoc.page_count):
            # If required for specific pages
            if pages:
                if str(pg) not in pages:
                    continue
            # Select the page
            page = pdfDoc[pg]
            # Get page text
            page_text = page.get_text("text")
            # Extract custom NER entities from page text
            custom_entities = extract_custom_ner_entities(page_text)
            # Perform redaction based on extracted custom entities
            matches_found = redact_matching_data(page, custom_entities)
            total_matches += matches_found
        
        print(f"{total_matches} Match(es) Found of PII Attributes In Input File: {input_file}")
        # Save to output
        pdfDoc.save(output_buffer)
        pdfDoc.close()
        # Save the output buffer to the output file
        with open(output_file, mode='wb') as f:
            f.write(output_buffer.getbuffer())

    elif file_ext == ".docx":
        doc = docx.Document(input_file)

        total_matches = 0
        for paragraph in doc.paragraphs:
            text = paragraph.text
            custom_entities = extract_custom_ner_entities(text)
            matches_found = redact_matching_data(paragraph, custom_entities)
            total_matches += matches_found

        print(f"{total_matches} Match(es) Found of PII Attributes In Input File: {input_file}")
        doc.save(output_file)

    else:
        print(f"Unsupported file format: {file_ext}")

def redact_files_in_directory(input_directory, output_directory):
    """
    Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
    """
    os.makedirs(output_directory, exist_ok=True)
    files = glob.glob(os.path.join(input_directory, "*.*"))

    for file in files:
        file_name = os.path.basename(file)
        output_file = os.path.join(output_directory, file_name)

        process_data(file, output_file)
        print(f"Redacted file saved: {output_file}")

# Redact custom NER entities in all PDF and DOCX files in the input directory and save the redacted files in the output directory
redact_files_in_directory(input_directory, output_directory)

In [ ]:
#display text from pdf or docx
import glob
from bs4 import BeautifulSoup
import spacy
from spacy import displacy
from docx import Document
import fitz

# Set the path to the directory containing the files
mypath = '/content/drive/MyDrive/customPIIredaction-main/synthetic_resumes1/'

# Load the spacy model
nlp = spacy.load("./model-last")

def SpacyMultiDoc(mypath):
    custom_ner_entities = []
    for file in glob.glob(mypath + "/*.*"):
        file_ext = file.lower().split(".")[-1]
        
        if file_ext == 'pdf':
            doc = fitz.open(file)
            text = ""
            for page in doc:
                text += page.get_text()
            doc.close()
            
        elif file_ext == 'docx':
            doc = Document(file)
            paragraphs = [p.text for p in doc.paragraphs]
            text = ' '.join(paragraphs)
        
        else:
            continue
        
        text = text.replace('\n', ' ')
        text = text.replace('\t', ' ')
        text = text.replace('\r', ' ')
        text = text.replace('\xa0', ' ')
        text = ' '.join(text.split())
        
        doc = nlp(text)
        displacy.render(doc, style="ent", jupyter=True)
        entities = [(ent.label_, ent.text) for ent in doc.ents]
        for ent in doc.ents:
            custom_ner_entities.append((ent.label_, ent.text))
    
    return custom_ner_entities

custom_ner_entities = SpacyMultiDoc(mypath)
print(custom_ner_entities)


In [ ]:
#create pdf to text merge all files
import os
from PyPDF2 import PdfReader

def pdf_to_text(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()

    return text

def convert_directory_to_text(directory_path):
    all_text = ""
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory_path, filename)
            text = pdf_to_text(pdf_path)
            all_text += text

    # Save merged text to a single file
    merged_output_path = os.path.join(directory_path, "merged_text.txt")
    with open(merged_output_path, 'w', encoding='utf-8') as merged_output_file:
        merged_output_file.write(all_text)


# Provide the directory path where the PDF files are located
directory_path = '/content/drive/MyDrive/customPIIredaction-main/synthetic_resumes1'

convert_directory_to_text(directory_path)


In [ ]:
# either through pdf or text
import os
import fitz
import docx

def pdf_to_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()

    return text

def docx_to_text(docx_path):
    doc = docx.Document(docx_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"

    return text

def convert_directory_to_text(directory_path):
    all_text = ""
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        if filename.endswith(".pdf"):
            text = pdf_to_text(file_path)
        elif filename.endswith(".docx"):
            text = docx_to_text(file_path)
        else:
            continue

        all_text += text

    # Save merged text to a single file
    merged_output_path = os.path.join(directory_path, "merged_text.txt")
    with open(merged_output_path, 'w', encoding='utf-8') as merged_output_file:
        merged_output_file.write(all_text)

# Provide the directory path where the files are located
directory_path = '/content/drive/MyDrive/customPIIredaction-main/synthetic_resumes1'

convert_directory_to_text(directory_path)
